# Mobile App for Lottery Addiction

* **What the project is about?**  
A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.  

* **What our goal is in this project.**  
For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?
* And much more...

In [62]:
# Since we are going to be using permutations and combinations it is a
# good idea to start by writing a factorial and combinatorics functions.

In [63]:
def factorial(n):
    if n == 1 or n == 0:
        return 1
    return n*factorial(n-1)

In [64]:
def combinations(n, k):
    if n == k:
        return 1
    numerator = factorial(n)
    denominator = factorial(n-k)*factorial(k)
    return numerator/denominator

## Winning big jackpot with single ticket

In [160]:
def one_ticket_probability(lot_num):
    total_outcomes = combinations(49, 6)
    succ_outcome = 1
    win_prob = succ_outcome/total_outcomes
    percent_prob = win_prob*100
    pretty_prob = "Your have {} in {} chance of winning this lottery OR {}%"
    return pretty_prob.format(succ_outcome, int(total_outcomes), round(percent_prob, 7))

In [66]:
one_ticket_probability([1,2,3, 5, 34])

'Your have 1 in 13983816 chance of winnig this lottery OR 7.2e-06%'

Created a functions that takes user ticket numbers as a list and then return the probability of winning the big jackpot(ie- all six numbers match) with a single ticket, in a way that is easier to read by the user.

## Have I ever won before?

We already wrote a function that can tell users what is the probability of winning the big prize with a single ticket. For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

In [67]:
import pandas as pd
hist_data = pd.read_csv("649.csv", parse_dates=["DRAW DATE"])

In [68]:
hist_data.head(5)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,1982-06-12,3,11,12,14,41,43,13
1,649,2,0,1982-06-19,8,33,36,37,39,41,9
2,649,3,0,1982-06-26,1,6,23,24,27,39,34
3,649,4,0,1982-07-03,3,9,10,13,20,43,34
4,649,5,0,1982-07-10,5,14,21,31,34,47,45


In [69]:
hist_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 11 columns):
PRODUCT            3665 non-null int64
DRAW NUMBER        3665 non-null int64
SEQUENCE NUMBER    3665 non-null int64
DRAW DATE          3665 non-null datetime64[ns]
NUMBER DRAWN 1     3665 non-null int64
NUMBER DRAWN 2     3665 non-null int64
NUMBER DRAWN 3     3665 non-null int64
NUMBER DRAWN 4     3665 non-null int64
NUMBER DRAWN 5     3665 non-null int64
NUMBER DRAWN 6     3665 non-null int64
BONUS NUMBER       3665 non-null int64
dtypes: datetime64[ns](1), int64(10)
memory usage: 315.0 KB


In [70]:
hist_data.head(5)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,1982-06-12,3,11,12,14,41,43,13
1,649,2,0,1982-06-19,8,33,36,37,39,41,9
2,649,3,0,1982-06-26,1,6,23,24,27,39,34
3,649,4,0,1982-07-03,3,9,10,13,20,43,34
4,649,5,0,1982-07-10,5,14,21,31,34,47,45


In [71]:
def extract_numbers(row):
    user_ticket = set()
    user_ticket.add(row["NUMBER DRAWN 1"])
    user_ticket.add(row["NUMBER DRAWN 2"])
    user_ticket.add(row["NUMBER DRAWN 3"])
    user_ticket.add(row["NUMBER DRAWN 4"])
    user_ticket.add(row["NUMBER DRAWN 5"])
    user_ticket.add(row["NUMBER DRAWN 6"])
    return user_ticket

In [72]:
hist_data["winning_ticket"] = hist_data.iloc[:, 4:10].apply(extract_numbers, axis=1)

In [161]:
def check_historical_occurence(user_ticket, winning_ticket):
#     user_ticket is a list
#     winning_ticket is a pandas series contaning all the past winnings
    user_ticket = set(user_ticket)
    win_count = 0
    for i in winning_ticket:
        if i == user_ticket:
            win_count += 1
    chances = one_ticket_probability(user_ticket)
    ever_won = "Your ticket combination has won {} time/s from 1982-2018".format(win_count)
    return chances + "\n" + ever_won

In [162]:
print(check_historical_occurence([1, 6, 39, 23, 24, 7], hist_data["winning_ticket"]))

Your have 1 in 13983816 chance of winning this lottery OR 7.2e-06%
Your ticket combination has won 0 time/s from 1982-2018


In [163]:
print(check_historical_occurence([1, 6, 39, 23, 24, 27], hist_data["winning_ticket"]))

Your have 1 in 13983816 chance of winning this lottery OR 7.2e-06%
Your ticket combination has won 1 time/s from 1982-2018


## Winning with multiple tickets

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — on this screen, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

In [180]:
def multi_ticket_probability(n_tickets):
    total_outcomes = combinations(49, 6)
    succ_outcomes = n_tickets
    win_prob = succ_outcomes/total_outcomes
    percent_prob = win_prob*100
    pretty_prob = "Your have {} in {} chance OR {} percent chance of winning this lottery ."
    return pretty_prob.format(succ_outcomes, int(total_outcomes), round(percent_prob, 7))

In [189]:
multi_ticket_probability(6991908)

'Your have 6991908 in 13983816 chance OR 50.0 percent chance of winning this lottery .'

## Winning small with single ticket

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

In [215]:
def probability_less_6(win_nums):
#     win_nums-the number of numbers user thinks that may win
    total_outcomes = combinations(49, 6)
    succ_outcomes = combinations(6, win_nums) * (49-win_nums-1)
    return succ_outcomes/total_outcomes

In [216]:
probability_less_6(5)

1.8449899512407772e-05

In [219]:
probability_less_6(3)

6.436011457816664e-05